In [0]:
import requests
import pandas as pd
import datetime
import sqlite3

In [0]:
# Fetch Data from APIs
def fetch_order_book(exchange, market):
    urls = {
        'cex': 'https://cex.io/api/order_book/{market[0]}/{market[1]}/',
        'coinbase': 'https://api.pro.coinbase.com/products/{market}/book?level=2'
    }
    url = urls[exchange].format(market=market)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {'message': response.text}

In [0]:
#Transformation logic to extract $100k of bid and ask order book data for each exchange
def process_order_book(data, poll_type):
    orders = data[poll_type]
    total_value = 0
    limit = 100000
    order_book = []
    
    for order in orders:
        if len(order)==3:
            price,quantity, _ = order
        else:
            price,quantity = order
            
        price = float(price)
        quantity = float(quantity)
        order_value = price * quantity

        if total_value + order_value > limit:
            remaining_value = limit - total_value
            quantity = remaining_value / price
            order_book.append((price, quantity))
            break

        order_book.append((price, quantity))
        total_value += order_value

    return order_book

In [0]:
#Load data into the database to persist data in each run
def load_to_db(exchange, market, timestamp, bids, asks, db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    # cursor.execute('''DROP TABLE IF EXISTS order_books''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS order_books (
            exchange TEXT,
            market TEXT,
            poll_type TEXT,
            price REAL,
            quantity REAL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')

    for price, quantity in bids:
        cursor.execute('''
            INSERT INTO order_books (exchange, market, timestamp, poll_type, price, quantity)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (exchange, market, timestamp, 'bid', price, quantity))

    for price, quantity in asks:
        cursor.execute('''
            INSERT INTO order_books (exchange, market, timestamp, poll_type, price, quantity)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (exchange, market, timestamp, 'ask', price, quantity))

    conn.commit()
    conn.close()

In [0]:
#Main control function to run all the above major ETL processes
def etl():
    exchanges = {
        'cex': [['BTC','USD'], ['ETH','USD']],
        'coinbase': ['BTC-USD', 'ETH-USD']
    }
    db_file = '/Workspace/Users/sas8595@mavs.uta.edu/order_books.db'
    
    timestamp = str(datetime.datetime.now())
    for exchange in exchanges:
        for market in exchanges[exchange]:
            data = fetch_order_book(exchange, market)
            if exchange=='cex':
                market=market[0]+ '-' +market[1]
#             print(data)
            bids = process_order_book(data, 'bids')
            asks = process_order_book(data, 'asks')
            load_to_db(exchange, market, timestamp, bids, asks, db_file)

if __name__ == "__main__":
    etl()

In [0]:
#Calculate the average mid price for a given market
def cal_avg_mid_price(market):
    conn = sqlite3.connect('/Workspace/Users/sas8595@mavs.uta.edu/order_books.db')
    cursor = conn.cursor()
    
    query = '''
    SELECT 
        AVG((bid.price + ask.price) / 2) as mkt_mid_price 
    FROM 
        (select price 
        from order_books 
        where poll_type='bid' and market = ? 
        order by timestamp DESC LIMIT 1) AS bid,
        (select price 
        from order_books 
        where poll_type='ask' and market = ? 
        order by timestamp desc LIMIT 1) AS ask
    '''
    
    cursor.execute(query, (market, market))
    result = cursor.fetchone()
    conn.close()
    
    return result[0] if result else None

average_mid_price = cal_avg_mid_price('ETH-USD')
print(f'Average Mid Price for ETH-USD: {average_mid_price}')
average_mid_price = cal_avg_mid_price('BTC-USD')
print(f'Average Mid Price for BTC-USD: {average_mid_price}')


Average Mid Price for ETH-USD: 3731.095
Average Mid Price for BTC-USD: 69827.1


In [0]:
# Find best exchange to execute a $50k buy or sell order and best price
def get_best_exchange_for_order(market, order_type):
    conn = sqlite3.connect('/Workspace/Users/sas8595@mavs.uta.edu/order_books.db')
    cursor = conn.cursor()
    
    if order_type == 'buy':
        df=pd.read_sql_query('''
        -- Calculate the total cost for each ask price based on a $50k order.
WITH buy_order AS (
    SELECT 
        exchange,
        market,
        price AS ask_price,
        quantity,
        (quantity * price) AS total_cost,
        (50000 / price) AS required_quantity
    FROM 
        order_books
    WHERE 
        poll_type = 'ask' and market = ?
)

-- Select the best ask price and exchange where the quantity is sufficient for a $50k order.
SELECT 
    exchange,
    market,
    ask_price,
    required_quantity
FROM 
    buy_order
WHERE
    quantity >= required_quantity
ORDER BY 
    ask_price ASC
LIMIT 1;

        ''',conn, params=(market,))
    else: 
        df=pd.read_sql_query('''
        -- Calculate the total revenue for each bid price based on a $50k order.
WITH sell_order AS (
    SELECT 
        exchange,
        market,
        price AS bid_price,
        quantity,
        (quantity * price) AS total_revenue,
        (50000 / price) AS required_quantity
    FROM 
        order_books
    WHERE 
        poll_type = 'bid' and market=?
)

-- Select the best bid price and exchange where the quantity is sufficient for a $50k order.
SELECT 
    exchange,
    market,
    bid_price,
    required_quantity
    
FROM 
    sell_order
WHERE
    quantity >= required_quantity
ORDER BY 
    bid_price DESC
LIMIT 1;
''',conn, params=(market,))
    
    conn.close()
    
    if not df.empty:
        return df
    else:
        return "None"


In [0]:
#Results for $50k buy orders BTC
best_exchange_for_50k_buy_btc_usd = get_best_exchange_for_order('BTC-USD', 'buy')
print(f'Best Exchange for $50k Buy Order for BTC:', end='\n')
best_exchange_for_50k_buy_btc_usd

Best Exchange for $50k Buy Order for BTC:


,exchange,market,ask_price,required_quantity
0,coinbase,BTC-USD,69711.22,0.717245


In [0]:
#Results for $50k sell orders BTC
best_exchange_for_50k_sell_btc_usd = get_best_exchange_for_order('BTC-USD', 'sell')
print(f'Best Exchange for $50k Sell Order for BTC:',end='\n')
best_exchange_for_50k_sell_btc_usd

Best Exchange for $50k Sell Order for BTC:


,exchange,market,bid_price,required_quantity
0,coinbase,BTC-USD,69711.21,0.717245


In [0]:
#Results for $50k buy orders ETH
best_exchange_for_50k_buy_eth_usd = get_best_exchange_for_order('ETH-USD', 'buy')
print(f'Best Exchange for $50k Buy Order for ETH:',end='\n')
best_exchange_for_50k_buy_eth_usd

Best Exchange for $50k Buy Order for ETH:
Out[10]: 'None'

In [0]:
#Results for $50k sell orders ETH
best_exchange_for_50k_sell_eth_usd = get_best_exchange_for_order('ETH-USD', 'sell')
print(f'Best Exchange for $50k Sell Order for ETH:',end='\n')
best_exchange_for_50k_sell_eth_usd

Best Exchange for $50k Sell Order for ETH:
Out[11]: 'None'